In [ ]:
import argparse
import os
from pathlib import Path

import albumentations as A
import pandas as pd
import pytorch_lightning as pl
import torch
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from classes import CustomDataset
from utils import *

In [ ]:
DATASET_PATHS = Path('celeba')
CHECKPOINT_PATH = Path('checkpoints')
BATCH_SIZE = 60
NUM_WORKERS = int(os.cpu_count() / 2)
SEED = 13


In [ ]:
train_df = pd.read_csv(DATASET_PATHS / 'train.csv')
test_df = pd.read_csv(DATASET_PATHS / 'test.csv')

train_data = []
test_data = []

for x in train_df.itertuples():
    train_data.append(x)
for x in test_df.itertuples():
    test_data.append(x)

print(f'Train total len: {len(train_data)}',
      f'Test total len: {len(test_data)}', sep='\n')

train, val = train_test_split(train_data, test_size=0.2, random_state=SEED)
test = test_data

print(f'Train shape: {len(train)}',
      f'Validation shape: {len(val)}',
      f'Test shape: {len(test)}', sep='\n')

pl.seed_everything(SEED)

torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

os.makedirs(CHECKPOINT_PATH, exist_ok=True)
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")


In [ ]:
train_transform = A.Compose([
    A.RandomGamma(),
    A.HorizontalFlip(p=0.5),
    A.Rotate(always_apply=False, limit=(-10, 10)),
    A.RandomBrightnessContrast(p=0.5),
    A.Resize(512, 512),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(512, 512),
    ToTensorV2(),
])

train_dataset = CustomDataset(train, train_transform)
valid_dataset = CustomDataset(val, test_transform)
test_dataset = CustomDataset(test, test_transform)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE,
                          drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                          drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)


In [ ]:
facenet_model, facenet_results = train_model(
    model_name="FaceNet",
    save_name = 'AugmentationAdd',
    model_hparams={"num_classes": 2},
    optimizer_name="Adam",
    optimizer_hparams={"lr": 0.0001},
    checkpoint_path=CHECKPOINT_PATH,
    device = device,
    gpu_id=[1],
    max_epochs= 1,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    seed=SEED,
)


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, drop_last=True, pin_memory=True, num_workers=NUM_WORKERS)
test_model = facenet_model
test_model.eval().to(device)

preds = []
targets = []
imgs = []

for image, label in test_loader:
    image = image[0][np.newaxis, ...].to(device)
    predict = test_model(image).argmax(dim=-1)
    # predict = model(image).to(device).cpu().numpy()
    predict = predict.detach().cpu().numpy()
    preds.append(predict.item())
    imgs.append(image)
    targets.append(label)


print(f'f1_score:{f1_score([x.item() for x in targets], preds)}')
print(f'precision:{precision_score([x.item() for x in targets], preds)}')
print(f'recall:{recall_score([x.item() for x in targets], preds)}')

In [ ]:
for x in test_model.parameters():
    print(x)